In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import warnings
import csv
warnings.filterwarnings('ignore')

In [3]:
def export_to_csv(filename):
    with open(filename + '.dat', 'r') as f, open(filename + '.csv', 'w') as out:
        csv_lines = [[field.strip() for field in line.split('|')] for line in f.readlines()]
        csv_lines = [line for line in csv_lines if len(line) == 6 and line[4] and line[4]]
        writer = csv.writer(out)
        writer.writerows(csv_lines)


In [4]:
export_to_csv('checkins')

In [5]:
df = pd.read_csv('checkins.csv')

In [6]:
cluster_centers_df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [10]:
subset = df[['latitude', 'longitude']]
X = subset.sample(100000)

In [11]:
clustering = MeanShift(bandwidth=0.1, n_jobs=-1)

In [12]:
%%time
clustering.fit_predict(X)

CPU times: user 53.8 s, sys: 13.8 s, total: 1min 7s
Wall time: 7min 22s


array([ 856,    7,   35, ..., 3134,   10,   23])

In [13]:
def remove_small_clusters(labels, threshold):
    return set([label for label in labels if labels.count(label) > threshold])

In [14]:
significant_clusters = remove_small_clusters(list(clustering.labels_), 15)

In [21]:
centres = clustering.cluster_centers_[list(significant_clusters)]

In [28]:
def make_line(row):
    return str(row[0]) + " " + str(row[1]) + '\n'

In [29]:
with open('centres.txt', 'w+') as f:
    f.writelines([make_line(row) for row in centres])

In [30]:
office_coordinates = [
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134],
]

In [31]:
def dist(centre):
    return min([((centre[0] - office[0]) ** 2 + (centre[1] - office[1]) ** 2) for office in office_coordinates])

In [37]:
dists = [(dist(centres[index]), index) for index in range(len(centres))]

In [44]:
def answer(question, ans):
    if ans is not str:
        ans = str(ans)
    print(ans)
    with open(question, 'w') as f:
        f.write(ans)

In [40]:
ans = [pair[1] for pair in sorted(dists)[:20]]

In [41]:
for index in ans:
    print(make_line(centres[index]))

-33.86250197551018 151.20327356734708

52.37285746744189 4.892427713953487

51.50228510165012 -0.12768248580858121

25.914752529333285 -80.27221166933333

33.81061799451611 -118.14348699838723

25.788474286463135 -80.21604512692939

26.006433456969717 -80.20848875333326

33.89873397944455 -118.06382277611141

33.87351363879097 -118.36903511763248

26.136623837499986 -80.35154221666669

26.103766230960886 -80.16065577686835

33.81794949849883 -117.89350147242097

34.06477638437427 -118.2649913583805

33.67464664254954 -117.85321739156196

33.99045346276298 -117.92367230850135

34.06237745546733 -118.00314962838418

34.03418791300449 -118.4350075116584

26.22048151523815 -80.18662999619039

34.12874979512202 -118.12395377630669

34.125554598989574 -118.34530121125515



In [45]:
answer('ans.txt', '-33.86250197551018 151.20327356734708')

-33.86250197551018 151.20327356734708
